## origin_insee n'existe pas
Garder nouvel insee ?

* 416532


## Origin_insee est disponible et différent de nouvel insee

**360480**  
origin_insee != insee  
la commune avec origin_insee existe => garder l'ancienne commune ?

idem 762778


**982732**  
origin_insee est devenu une commune déléguée : https://www.insee.fr/fr/metadonnees/cog/commune/COM44059-fresnay-en-retz  
même problème avec les communes périmée  
origin_insee n'existe plus  
garder nouvel insee ?

idem pour 498361 (origin_insee=85060), 266457, 459259, 750210, 286296, 989990, 982732, 279671

Liens:
- check commune : https://www.insee.fr/fr/recherche?q=01154&debut=0

In [1]:
%load_ext sql
# connect to db
%sql postgresql://postgres:postgres@db:5432/postgres
from public_data.models import ZoneUrba, Commune
import json
from django.contrib.gis.geos import Polygon
from django.db.models import F
from ipyleaflet import Map, GeoJSON
from colour import Color, RGB_TO_COLOR_NAMES
import random

check = [
    416532,
    360480, 762778,
    982732, 498361, 266457, 459259, 750210, 286296, 989990, 982732, 279671
]

geojson = None

COLOR = {v[0]: f'rgb({",".join([str(_) for _ in k])})' for k,v in RGB_TO_COLOR_NAMES.items()}

In [2]:
m = Map(center=[46.603354, 1.888334], zoom=6)

def display_commune(insee, color=COLOR["HotPink"], fit=False):
    try:
        commune = Commune.objects.get(insee=insee)

        m.add_layer(
            GeoJSON(
                data=json.loads(commune.mpoly.geojson),
                style={'color': color, 'opacity': 0.9, 'weight': 3}
            )
        )
        
        if fit:
            bounds = commune.mpoly.extent
            m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
    except Commune.DoesNotExist:
        print(f"commune does not exists {insee}")

def handle_feature_click(event, feature, **kwargs):
    print(feature["properties"])
    display_commune(feature["properties"]["insee"], fit=True)

    origin_insee = feature["properties"]["origin_insee"]
    if origin_insee:
        display_commune(origin_insee,color=COLOR["SeaGreen"])

In [3]:
if geojson:
    m.remove_layer(geojson)

geojson_data = {
    'type': 'FeatureCollection',
    'features': [
        json.loads(zone_urba.mpoly.geojson)  | {
            "properties": {
                "id": zone_urba.id,
                "insee": zone_urba.insee,
                "origin_insee": zone_urba.origin_insee,
            }
        }
        for zone_urba in ZoneUrba.objects.filter(id__in=check)
    ]
}
geojson = GeoJSON(
    data=geojson_data,
    style={'color': COLOR["Firebrick"], 'opacity': 0.7, 'weight': 3},
)
geojson.on_click(handle_feature_click)
m.add_layer(geojson)

In [4]:
display(m)

Map(center=[46.603354, 1.888334], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

{'id': 762778, 'insee': '18062', 'origin_insee': '18215', 'style': {'color': 'rgb(178,34,34)', 'opacity': 0.7, 'weight': 3}}


In [5]:
zone = ZoneUrba.objects.get(id=279671)
center = zone.mpoly.centroid
m.center = [center.y, center.x]

In [ ]:
qs = ZoneUrba.objects.all()
print(f"Zones: {qs.count()}")
print(f"Zones sans code INSEE initialement: {qs.filter(origin_insee__isnull=True).count()}")
qs = qs.exclude(origin_insee__isnull=True)
print(f"Zones sans changement de code INSEE: {qs.filter(origin_insee=F('insee')).count()}")

Zones: 1010442


In [ ]:
ZoneUrba.objects.exclude(insee=F("origin_insee")).filter(origin_insee__isnull=True).count()

In [ ]:
%%sql
DROP TABLE IF EXISTS pdz_join_pdc;
CREATE TEMPORARY TABLE pdz_join_pdc AS
select pdz.id, pdz.origin_insee, pdc.id as commune_id, pdc.insee, ST_Area(ST_Transform(ST_Intersection(pdc.mpoly, pdz.mpoly), 2154)) as area
from public_data_zoneurba pdz 
inner join public_data_commune pdc on ST_Intersects(pdz.mpoly, pdc.mpoly);